In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

class WorkoutRecommendationSystem:
    def __init__(self):
        self.model = None
        self.bmi_status_encoder = None
        self.age_category_encoder = None
        self.workout_encoder = None
        self.workout_decoder = None
        self.scaler = None

    def train(self, dataset_path='dataset1.csv'):
        """Train the workout recommendation model"""
        print("Using TensorFlow version:", tf.__version__)

        # Load dataset
        print("Loading and preprocessing data...")
        df = pd.read_csv(dataset_path)

        # Check for missing values
        print("\nMissing values count:")
        print(df.isna().sum())

        # Data preprocessing
        # Replace NaN values with empty string for workout columns
        workout_cols = ['Workout1', 'Workout2', 'Workout3', 'Workout4', 'Workout5']
        df[workout_cols] = df[workout_cols].fillna('')

        # Create age category based on age ranges
        df['Kategori_Usia'] = df['Umur'].apply(self.get_age_category)

        # BMI status is already in the dataset as BMI_status

        # Check BMI distribution
        print("\nBMI status distribution:")
        print(df['BMI_status'].value_counts())
        print("\nAge category distribution:")
        print(df['Kategori_Usia'].value_counts())

        # Create encoders
        self.bmi_status_encoder = LabelEncoder()
        self.age_category_encoder = LabelEncoder()

        # Fit and transform categorical features
        df['BMI_status_Encoded'] = self.bmi_status_encoder.fit_transform(df['BMI_status'])
        df['Kategori_Usia_Encoded'] = self.age_category_encoder.fit_transform(df['Kategori_Usia'])

        # Print encoder mappings
        print("\nBMI Status Mapping:")
        for i, category in enumerate(self.bmi_status_encoder.classes_):
            print(f"{category}: {i}")

        print("\nAge Category Mapping:")
        for i, category in enumerate(self.age_category_encoder.classes_):
            print(f"{category}: {i}")

        # Scale numerical features
        self.scaler = MinMaxScaler()
        numerical_cols = ['tinggi_badan', 'berat_badan', 'Umur']
        # Add BMI calculation for consistency, even though BMI_status exists
        df['BMI'] = df['berat_badan'] / ((df['tinggi_badan'] / 100) ** 2)
        numerical_cols.append('BMI')
        df[numerical_cols] = self.scaler.fit_transform(df[numerical_cols])

        # Create a mapping of workout types
        unique_workouts = set()
        for col in workout_cols:
            unique_workouts.update(df[col].unique())
        unique_workouts.discard('')  # Remove empty string

        print(f"\nTotal unique workout types: {len(unique_workouts)}")
        print("Workout types:", sorted(unique_workouts))

        # Create workout encoder
        self.workout_encoder = {workout: i for i, workout in enumerate(unique_workouts)}
        self.workout_decoder = {i: workout for workout, i in self.workout_encoder.items()}

        # Create feature matrix X and target matrix Y
        X_features = df[['tinggi_badan', 'berat_badan', 'Umur', 'BMI',
                         'BMI_status_Encoded', 'Kategori_Usia_Encoded']].values

        Y = np.array([self._get_workout_encodings(row, workout_cols) for _, row in df.iterrows()])

        print(f"\nFeature matrix shape: {X_features.shape}")
        print(f"Target matrix shape: {Y.shape}")

        # Print sample count for each workout type
        print("\nSample counts for each workout type:")
        workout_counts = Y.sum(axis=0)
        for i, count in enumerate(workout_counts):
            if count > 0:  # Only show workouts that appear in the dataset
                print(f"{self.workout_decoder[i]}: {int(count)}")

        # Split the data
        X_train, X_test, Y_train, Y_test = train_test_split(X_features, Y, test_size=0.2, random_state=42)

        # Build and train model
        print("\nBuilding and training the model...")
        self.model = self._build_recommendation_model(X_features.shape[1], len(unique_workouts))

        # Print model summary
        print("\nModel Architecture:")
        self.model.summary()

        # Callbacks
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )

        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=0.00001
        )

        # Train the model
        history = self.model.fit(
            X_train, Y_train,
            epochs=50,
            batch_size=8,
            validation_data=(X_test, Y_test),
            verbose=1,
            callbacks=[early_stopping, reduce_lr]
        )

        # Evaluate model on test data
        test_loss, test_hamming_loss, test_f1, test_accuracy = self.model.evaluate(X_test, Y_test)
        print(f"\nTest Loss: {test_loss:.4f}")
        print(f"Test Hamming Loss: {test_hamming_loss:.4f}")
        print(f"Test F1 Score: {test_f1:.4f}")
        print(f"Test Binary Accuracy: {test_accuracy:.4f}")

        # Test the model with examples
        self._test_model_examples()

        print("\nDone! The model has been trained successfully.")

        # Return metrics for easier access
        return {
            'test_loss': test_loss,
            'test_hamming_loss': test_hamming_loss,
            'test_f1': test_f1,
            'test_accuracy': test_accuracy
        }

    def _get_workout_encodings(self, row, workout_cols):
        """Convert row's workout data to one-hot encoding"""
        workout_encoding = np.zeros(len(self.workout_encoder))
        for col in workout_cols:
            if row[col] and row[col] in self.workout_encoder:
                workout_encoding[self.workout_encoder[row[col]]] = 1
        return workout_encoding

    def _build_recommendation_model(self, input_shape, num_workouts):
        """Build the neural network model"""
        # Input layer
        inputs = Input(shape=(input_shape,))

        # Hidden layers
        x = Dense(128, activation='relu')(inputs)
        x = Dropout(0.3)(x)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.3)(x)
        x = Dense(32, activation='relu')(x)
        x = Dropout(0.2)(x)

        # Output layer
        outputs = Dense(num_workouts, activation='sigmoid')(x)

        # Create and compile model
        model = Model(inputs=inputs, outputs=outputs)

        # Define custom metrics for multi-label classification
        def hamming_loss(y_true, y_pred):
            # Convert predictions to binary (0 or 1)
            threshold = 0.5
            y_pred_binary = tf.cast(tf.greater_equal(y_pred, threshold), tf.float32)

            # Calculate Hamming loss
            return tf.reduce_mean(tf.cast(tf.not_equal(y_true, y_pred_binary), tf.float32))

        def f1_score(y_true, y_pred):
            # Convert predictions to binary (0 or 1)
            threshold = 0.5
            y_pred_binary = tf.cast(tf.greater_equal(y_pred, threshold), tf.float32)

            # Calculate true positives, false positives, false negatives
            true_positives = tf.reduce_sum(y_true * y_pred_binary)
            false_positives = tf.reduce_sum((1 - y_true) * y_pred_binary)
            false_negatives = tf.reduce_sum(y_true * (1 - y_pred_binary))

            # Calculate precision and recall
            precision = true_positives / (true_positives + false_positives + tf.keras.backend.epsilon())
            recall = true_positives / (true_positives + false_negatives + tf.keras.backend.epsilon())

            # Calculate F1 score
            return 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=[hamming_loss, f1_score, tf.keras.metrics.BinaryAccuracy(threshold=0.3)]
        )

        return model

    def _test_model_examples(self):
        """Test the model with sample inputs"""
        print("\nTesting the model with sample inputs:")


                # Muda, Normal
        height1, weight1, age1 = 170, 65, 25  # Normal weight for a young adult
        recommendations1, num_workouts1, bmi_cat1, age_cat1 = self.recommend(height1, weight1, age1, include_details=True)
        print(f"\nFor person with height {height1}cm, weight {weight1}kg, age {age1}:")
        print(f"BMI Category: {bmi_cat1}, Age Category: {age_cat1}")
        print(f"Recommended {num_workouts1} workouts:")
        for i, (workout, score) in enumerate(recommendations1, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Muda, Underweight
        height2, weight2, age2 = 160, 45, 22  # Underweight for a young adult
        recommendations2, num_workouts2, bmi_cat2, age_cat2 = self.recommend(height2, weight2, age2, include_details=True)
        print(f"\nFor person with height {height2}cm, weight {weight2}kg, age {age2}:")
        print(f"BMI Category: {bmi_cat2}, Age Category: {age_cat2}")
        print(f"Recommended {num_workouts2} workouts:")
        for i, (workout, score) in enumerate(recommendations2, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Muda, Overweight
        height3, weight3, age3 = 175, 85, 25  # Overweight for a young adult
        recommendations3, num_workouts3, bmi_cat3, age_cat3 = self.recommend(height3, weight3, age3, include_details=True)
        print(f"\nFor person with height {height3}cm, weight {weight3}kg, age {age3}:")
        print(f"BMI Category: {bmi_cat3}, Age Category: {age_cat3}")
        print(f"Recommended {num_workouts3} workouts:")
        for i, (workout, score) in enumerate(recommendations3, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Muda, Obesitas
        height4, weight4, age4 = 170, 95, 25  # Obesity for a young adult
        recommendations4, num_workouts4, bmi_cat4, age_cat4 = self.recommend(height4, weight4, age4, include_details=True)
        print(f"\nFor person with height {height4}cm, weight {weight4}kg, age {age4}:")
        print(f"BMI Category: {bmi_cat4}, Age Category: {age_cat4}")
        print(f"Recommended {num_workouts4} workouts:")
        for i, (workout, score) in enumerate(recommendations4, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Dewasa, Normal
        height5, weight5, age5 = 170, 75, 40  # Normal weight for an adult
        recommendations5, num_workouts5, bmi_cat5, age_cat5 = self.recommend(height5, weight5, age5, include_details=True)
        print(f"\nFor person with height {height5}cm, weight {weight5}kg, age {age5}:")
        print(f"BMI Category: {bmi_cat5}, Age Category: {age_cat5}")
        print(f"Recommended {num_workouts5} workouts:")
        for i, (workout, score) in enumerate(recommendations5, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Dewasa, Underweight
        height6, weight6, age6 = 160, 50, 40  # Underweight for an adult
        recommendations6, num_workouts6, bmi_cat6, age_cat6 = self.recommend(height6, weight6, age6, include_details=True)
        print(f"\nFor person with height {height6}cm, weight {weight6}kg, age {age6}:")
        print(f"BMI Category: {bmi_cat6}, Age Category: {age_cat6}")
        print(f"Recommended {num_workouts6} workouts:")
        for i, (workout, score) in enumerate(recommendations6, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Dewasa, Overweight
        height7, weight7, age7 = 175, 90, 40  # Overweight for an adult
        recommendations7, num_workouts7, bmi_cat7, age_cat7 = self.recommend(height7, weight7, age7, include_details=True)
        print(f"\nFor person with height {height7}cm, weight {weight7}kg, age {age7}:")
        print(f"BMI Category: {bmi_cat7}, Age Category: {age_cat7}")
        print(f"Recommended {num_workouts7} workouts:")
        for i, (workout, score) in enumerate(recommendations7, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Dewasa, Obesitas
        height8, weight8, age8 = 170, 100, 40  # Obesity for an adult
        recommendations8, num_workouts8, bmi_cat8, age_cat8 = self.recommend(height8, weight8, age8, include_details=True)
        print(f"\nFor person with height {height8}cm, weight {weight8}kg, age {age8}:")
        print(f"BMI Category: {bmi_cat8}, Age Category: {age_cat8}")
        print(f"Recommended {num_workouts8} workouts:")
        for i, (workout, score) in enumerate(recommendations8, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Tua, Underweight
        height9, weight9, age9 = 160, 45, 65  # Underweight for an older adult
        recommendations9, num_workouts9, bmi_cat9, age_cat9 = self.recommend(height9, weight9, age9, include_details=True)
        print(f"\nFor person with height {height9}cm, weight {weight9}kg, age {age9}:")
        print(f"BMI Category: {bmi_cat9}, Age Category: {age_cat9}")
        print(f"Recommended {num_workouts9} workouts:")
        for i, (workout, score) in enumerate(recommendations9, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Tua, Normal
        height10, weight10, age10 = 165, 70, 65  # Normal weight for an older adult
        recommendations10, num_workouts10, bmi_cat10, age_cat10 = self.recommend(height10, weight10, age10, include_details=True)
        print(f"\nFor person with height {height10}cm, weight {weight10}kg, age {age10}:")
        print(f"BMI Category: {bmi_cat10}, Age Category: {age_cat10}")
        print(f"Recommended {num_workouts10} workouts:")
        for i, (workout, score) in enumerate(recommendations10, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Tua, Overweight
        height11, weight11, age11 = 165, 85, 65  # Overweight for an older adult
        recommendations11, num_workouts11, bmi_cat11, age_cat11 = self.recommend(height11, weight11, age11, include_details=True)
        print(f"\nFor person with height {height11}cm, weight {weight11}kg, age {age11}:")
        print(f"BMI Category: {bmi_cat11}, Age Category: {age_cat11}")
        print(f"Recommended {num_workouts11} workouts:")
        for i, (workout, score) in enumerate(recommendations11, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")

        # Tua, Obesitas
        height12, weight12, age12 = 160, 95, 65  # Obesity for an older adult
        recommendations12, num_workouts12, bmi_cat12, age_cat12 = self.recommend(height12, weight12, age12, include_details=True)
        print(f"\nFor person with height {height12}cm, weight {weight12}kg, age {age12}:")
        print(f"BMI Category: {bmi_cat12}, Age Category: {age_cat12}")
        print(f"Recommended {num_workouts12} workouts:")
        for i, (workout, score) in enumerate(recommendations12, 1):
            print(f"{i}. {workout} (confidence: {score:.4f})")



    def get_bmi_category(self, bmi):
        """Determine BMI category from BMI value"""
        if bmi < 18.5:
            return 'underweight'
        elif bmi < 25:
            return 'normal'
        elif bmi < 30:
            return 'overweight'
        else:
            return 'obesitas'

    def get_age_category(self, age):
        """Determine age category from age value"""
        if age < 36:
            return 'Muda'
        elif age < 61:
            return 'Dewasa'
        else:
            return 'Tua'

    def recommend(self, height, weight, age, include_details=False):
        """
        Recommend workouts based on height, weight, and age

        Parameters:
        - height: Height in cm
        - weight: Weight in kg
        - age: Age in years
        - include_details: If True, return scores and categories

        Returns:
        - If include_details=False: List of recommended workouts, number of recommended workouts
        - If include_details=True: List of (workout, score) tuples, number of workouts, BMI category, age category
        """
        # Calculate BMI
        height_m = height / 100
        bmi = weight / (height_m ** 2)

        # Get categories
        bmi_category = self.get_bmi_category(bmi)
        age_category = self.get_age_category(age)

        # Encode categories
        try:
            bmi_encoded = self.bmi_status_encoder.transform([bmi_category])[0]
            age_encoded = self.age_category_encoder.transform([age_category])[0]
        except:
            print(f"Warning: Could not encode BMI category '{bmi_category}' or age category '{age_category}'.")
            print("Using default encoding values.")
            bmi_encoded = 0
            age_encoded = 0

        # Scale features
        features = np.array([[height, weight, age, bmi]])
        scaled_features = self.scaler.transform(features)

        # Create input array
        input_data = np.array([[
            scaled_features[0][0],  # Height
            scaled_features[0][1],  # Weight
            scaled_features[0][2],  # Age
            scaled_features[0][3],  # BMI
            bmi_encoded,            # BMI category
            age_encoded             # Age category
        ]])

        # Get predictions
        predictions = self.model.predict(input_data)[0]

        # Determine number of workouts based on age category
        # (following the pattern from the dataset)
        if age_category == 'Muda':
            num_workouts = 5  # Young gets 5 workouts
        elif age_category == 'Dewasa':
            num_workouts = 4  # Adults get 4 workouts
        else:  # Tua (Elderly)
            num_workouts = 3  # Elderly get 3 workouts

        # Get top workout indices
        top_indices = predictions.argsort()[-num_workouts:][::-1]

        if include_details:
            # Return workouts with confidence scores and categories
            recommendations = [(self.workout_decoder[idx], predictions[idx]) for idx in top_indices]
            return recommendations, num_workouts, bmi_category, age_category
        else:
            # Return only workout names
            recommendations = [self.workout_decoder[idx] for idx in top_indices]
            return recommendations, num_workouts

    def interactive_mode(self):
        """Run the system in interactive mode for making recommendations"""
        print("\nWorkout Recommendation System")
        print("-----------------------------")

        try:
            while True:
                try:
                    height = float(input("Enter your height (cm): "))
                    weight = float(input("Enter your weight (kg): "))
                    age = int(input("Enter your age: "))

                    # Get recommendations with details
                    recommendations, num_workouts, bmi_cat, age_cat = self.recommend(height, weight, age, include_details=True)

                    # Display results
                    print(f"\nBased on your data:")
                    print(f"- Height: {height}cm")
                    print(f"- Weight: {weight}kg")
                    print(f"- Age: {age}")
                    print(f"- BMI: {weight / ((height/100)**2):.2f} ({bmi_cat})")
                    print(f"- Age Category: {age_cat}")
                    print(f"\nWe recommend the following {num_workouts} workouts:")

                    for i, (workout, score) in enumerate(recommendations, 1):
                        print(f"{i}. {workout} (confidence: {score:.4f})")

                    another = input("\nWant to try another recommendation? (y/n): ")
                    if another.lower() != 'y':
                        break

                except ValueError:
                    print("Error: Please enter valid numeric values.")
                    continue

        except KeyboardInterrupt:
            print("\nExiting interactive mode.")
        except Exception as e:
            print(f"An error occurred: {e}")


def main():
    """Main function to run the system"""
    print("Workout Recommendation System")
    print("=============================")

    dataset_path = 'dataset1.csv'

    print(f"Using dataset: {dataset_path}")

    # Create and train the system
    system = WorkoutRecommendationSystem()
    metrics = system.train(dataset_path=dataset_path)

    print("\nFinal model metrics:")
    print(f"- Loss: {metrics['test_loss']:.4f}")
    print(f"- Hamming Loss: {metrics['test_hamming_loss']:.4f}")
    print(f"- F1 Score: {metrics['test_f1']:.4f}")
    print(f"- Binary Accuracy: {metrics['test_accuracy']:.4f}")

    # Run interactive mode
    system.interactive_mode()


if __name__ == "__main__":
    main()

Workout Recommendation System
Using dataset: dataset1.csv
Using TensorFlow version: 2.18.0
Loading and preprocessing data...

Missing values count:
Umur                0
BMI_status          0
tinggi_badan        0
berat_badan         0
Workout1            0
Workout2            0
Workout3            0
Workout4          310
Workout5          611
banyak_workout      0
dtype: int64

BMI status distribution:
BMI_status
normal         301
underweight    266
obesitas       241
overweight     126
Name: count, dtype: int64

Age category distribution:
Kategori_Usia
Tua       453
Dewasa    283
Muda      198
Name: count, dtype: int64

BMI Status Mapping:
normal: 0
obesitas: 1
overweight: 2
underweight: 3

Age Category Mapping:
Dewasa: 0
Muda: 1
Tua: 2

Total unique workout types: 12
Workout types: ['assisted pull up', 'assisted push up', 'berenang', 'bersepeda', 'bersepeda santai', 'dips', 'jogging', 'jogging santai', 'jump squat', 'pull up', 'push up', 'squat']

Feature matrix shape: (934, 6)
Tar

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 12)             │           396 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,628 (45.42 KB)

 Trainable params: 11,628 (45.42 KB)

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

For person with height 165cm, weight 70kg, age 65:
BMI Category: overweight, Age Category: Tua
Recommended 3 workouts:
1. assisted pull up (confidence: 0.9998)
2. push up (confidence: 0.9991)
3. bersepeda santai (confidence: 0.5797)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

For person with height 165cm, weight 85kg, age 65:
BMI Category: obesitas, Age Category: Tua
Recommended 3 workouts:
1. assisted push up (confidence: 1.0000)
2. squat (confidence: 1.0000)
3. berenang (confidence: 0.4599)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(



For person with height 160cm, weight 95kg, age 65:
BMI Category: obesitas, Age Category: Tua
Recommended 3 workouts:
1. assisted push up (confidence: 1.0000)
2. squat (confidence: 1.0000)
3. berenang (confidence: 0.8770)

Done! The model has been trained successfully.

Final model metrics:
- Loss: 0.1631
- Hamming Loss: 0.0911
- F1 Score: 0.8598
- Binary Accuracy: 0.8832

Workout Recommendation System
-----------------------------
Enter your height (cm): 177
Enter your weight (kg): 77
Enter your age: 17
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(



Based on your data:
- Height: 177.0cm
- Weight: 77.0kg
- Age: 17
- BMI: 24.58 (normal)
- Age Category: Muda

We recommend the following 5 workouts:
1. pull up (confidence: 0.9973)
2. jogging (confidence: 0.8638)
3. push up (confidence: 0.8387)
4. squat (confidence: 0.7676)
5. bersepeda (confidence: 0.6210)

Want to try another recommendation? (y/n): n
